In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz
import pytesseract
import io
import sys

from pprint import pprint
from PIL import Image
from pdf2image import convert_from_path
from platform import system
from difflib import get_close_matches

import util
import lines
import group
import label
import records
import extract
import date

In [2]:
test_files_dir = ".././test_files/"
test_dfs_dir = test_files_dir + "tesseract_data_frames/"
test_output_dir = test_files_dir + "extracted_indexes/"
test_files = util.list_files(test_files_dir, suffix=".pdf")
test_dfs = util.list_files(test_dfs_dir)
file = test_dfs[0]

indexes_path = "/home/jeusa/Files/Documents/Indexex/ILO_yearly_indices/tesseract_data_frames/"
output_dir = "/home/jeusa/Files/Documents/Indexex/ILO_yearly_indices/extracted_indexes/"

if system()=="Windows":
    indexes_path = "H:\Documents\Indexex\ILO_yearly_indices\\tesseract_data_frames"
    output_dir = "H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes"

indexes_csv = util.list_files(indexes_path, suffix=".csv")

display(test_dfs, file)

['.././test_files/tesseract_data_frames/LS_index_1921.csv',
 '.././test_files/tesseract_data_frames/LS_index_1944.csv',
 '.././test_files/tesseract_data_frames/LS_index_1968.csv',
 '.././test_files/tesseract_data_frames/LS_index_1986_split.csv',
 '.././test_files/tesseract_data_frames/LS_index_1976.csv',
 '.././test_files/tesseract_data_frames/LS_index_1986.csv',
 '.././test_files/tesseract_data_frames/LS_index_1920.csv',
 '.././test_files/tesseract_data_frames/LS_index_1985-1.csv']

'.././test_files/tesseract_data_frames/LS_index_1921.csv'

In [13]:
pdf_file = test_files[8]
display(test_files, pdf_file)

['.././test_files/LS_index_1920.pdf',
 '.././test_files/LS_index_1937.pdf',
 '.././test_files/Argentinia_LS_index_1934_test_I_DE.pdf',
 '.././test_files/LS_index_1986.pdf',
 '.././test_files/LS_index_1976.pdf',
 '.././test_files/LS_index_1944_split.pdf',
 '.././test_files/LS_index_1944.pdf',
 '.././test_files/LS_index_1986_split.pdf',
 '.././test_files/LS_index_1921.pdf',
 '.././test_files/LS_index_1985-1.pdf',
 '.././test_files/LS_index_1920_split.pdf',
 '.././test_files/LS_index_1968.pdf']

'.././test_files/LS_index_1921.pdf'

In [14]:
# fitz mode
start_page = 1

pdf_words, pdf_dicts = util.read_pdf(pdf_file, start_page)

w_df = lines.make_words_df(pdf_words, start_page)

w = w_df.rename(columns={"text": "line_text"})
w["x0"] = [round(x, 2) for x in w["x0"]]
w["y0"] = [round(x, 2) for x in w["y0"]]
w["x1"] = [round(x, 2) for x in w["x1"]]
w["y1"] = [round(x, 2) for x in w["y1"]]

w_l = lines.merge_close_lines(w)
w_l = lines.remove_useless_lines(w_l)

l_d = lines.make_lines_df_from_dicts(pdf_dicts, start_page)
l = lines.merge_close_lines(l_d)
l = lines.remove_useless_lines(l)

r_df = extract.extract_indexes(w_df, w_l, pdf_file, "fitz")
r_df

Reading pdf from .././test_files/LS_index_1921.pdf
...
Finished reading 12 page(s)


,country,region,text,page,extracted_date,extracted_day,extracted_month,extracted_year,full_text
date_type,,,,,,,,,
1,10,10,10,10,10,10,10,10,10
-1,4,4,4,4,4,4,4,4,4


,country,region,text,date,year,page,extracted_date,full_text
0,ENGLISH EDITION OF THE LEGISLATIVE SERIES OF,text an extract a summary or the title only of...,"the Legislative Series for 1921, which may be ...",,,3,,"the Legislative Series for 1921, which may be ..."
1,INTERNATIONAL,,,9.2.,1921,3,Feb. 9th,Feb. 9th.
2,INTERNATIONAL,,,14.2.,1921,3,Feb. 14th,Feb. 14th
3,INTERNATIONAL,,i6th,4.3.,1921,3,Mar. 4,Mar. 4/i6th
4,INTERNATIONAL,,,8.10.,1921,3,Oct. 8th,Oct. 8th
...,...,...,...,...,...,...,...,...
138,T U N I S,,"Order of the Director-General of Agriculture, ...",25.4.,1921,12,Apr. 25th,Apr. 25th Order of the Director-General of Agr...
139,T U N I S,,"Order of the Director-General of Public Works,...",16.7.,1921,12,July 16th,July 16th Order of the Director-General of Pub...
140,UNITED STATES OF AMERICA,,Act to limit the immigration of aliens into th...,19.5.,1921,12,May 19th,May 19th Act to limit the immigration of alien...
141,UNITED STATES OF AMERICA,,Act for the promotion of the welfare and hygie...,23.11.,1921,12,Nov. 23rd,Nov. 23rd Act for the promotion of the welfare...


In [21]:
b_x0, b_x1, c0_c = group.group_line_starts_ends(l, "fitz")
bor = lines.make_borders_df(b_x0, b_x1)

In [16]:
# tess mode
pdf_df = pd.read_csv(file)
lines_df = lines.make_lines_df_from_ocr(pdf_df)
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df, "tess")
borders = lines.make_borders_df(bins_x0, bins_x1)

rec_df = extract.extract_indexes(pdf_df, lines_df, file, "tess")
rec_df

,country,region,text,page,extracted_date,extracted_day,extracted_month,extracted_year,full_text
date_type,,,,,,,,,
1,13,13,13,13,13,13,13,13,13
-1,2,2,2,2,2,2,2,2,2
3,1,1,1,1,1,1,1,1,1


,country,region,text,date,year,page,extracted_date,full_text
0,ENGLISH EDITION OF THE LEGISLATIVE SERIES OF,text an extract a summary or the title only of...,"the Legislative Series for 1921, which may be ...",,,3,,"the Legislative Series for 1921, which may be ..."
1,ENTERNATIONAL,,Convention respecting insurance against oc- cu...,9.2.,1921,3,Feb. 9th,Feb. 9th. Convention respecting insurance agai...
2,ENTERNATIONAL,,Convention between France and Belgium for the ...,14.2.,1921,3,Feb. 14th,Feb. 14th Convention between France and Belgiu...
3,ENTERNATIONAL,,16th Agreecinent respecting the treatment of I...,4.3.,1921,3,Mar. 4,Mar. 4/16th Agreecinent respecting the treatme...
4,ENTERNATIONAL,,Convention between Italy and the United States...,8.10.,1921,3,Oct. 8th,Oct. 8th Convention between Italy and the Unit...
...,...,...,...,...,...,...,...,...
152,SWITZERLAND,Tunlis,"Order of the Director-General of Agriculture, ...",25.4.,1921,12,Apr. 25th,Apr. 25th Order of the Director-General of Agr...
153,SWITZERLAND,Tunlis,"Order of the Director-General of Public Works,...",16.7.,1921,12,July 16th,July 16th Order of the Director-General of Pub...
154,Unitep STATES OF AMERICA,,Act to limit the immigration of aliens into th...,19.5.,1921,12,May 19th,May 19th Act to limit the immigration of alien...
155,Unitep STATES OF AMERICA,,Act for the promotion of the welfare and hygie...,23.11.,1921,12,Nov. 23rd,Nov. 23rd Act for the promotion of the welfare...


## Probleme
- Dokumente mit 2 Spalten FIXED
- Dokumente mit Länder-Überschrift über ganze Breite der Seite IMPROVED
- Länder- und Regionenspalte vereinen?
    - Regionenerkennung oft nicht richtig, weil Groß- und Kleinbuchstaben vom OCR nicht richtig erkannt werden
- Dokumente/Seiten mit vielen Artifakten
    - 1944
        - S. 8-9, schlechte Ländererkennung FIXED
            - S. 25 falsches Labeling FIXED
        - S. 28 Inhalt fehlt? TESSERACT PROBLEM
        - S. 10-11, schlechte Ländererkennung FIXED
- Date-Type-Erkennung fehlerhaft
    - 1920 FIXED
- 1920 Seite 11 Uruguay
    - unvollständige Datums-Erkennung FIXED
    

## Todo
- doppelseitige Extraktion funktioniert auch im fitz-Modus FAILED
- Kommandozeilenprogramm